## Problem B: Food Production Revisited

This question is a modification of the sample problem from Session 9, and it incoporates a limit on how long oil can be stored. 

A food factory uses a certain amount of canola oil every month as a raw ingredient. The price of canola oil fluctuates from month to month, as well as the amount of oil that is used each month for production. The prices and oil required for the next six months are as follows

|Month | 1 | 2 | 3 | 4 | 5 | 6 |
|--|--|--|--|--|--|--|
| Price (\$) per ton | 150 | 160 | 180 | 170 | 180 | 160 |
| Tons of oil used | 2000 | 2500 | 3000 | 3000 | 2500 | 2000 |

The factory's supplier for canola oil delivers it on the first day of every month, and charges the prices above. The factory can decide how much oil to buy each month from the supplier.  At the end of each month, the factory can also store unused oil for future use, but the inventory of oil (the total amount stored) cannot exceed **8000** tons at any given time. (The current inventory of oil before the shipment in Month 1 is zero.) **Moreover, oil be stored for at most 2 months. For example, oil bought in Month 1 can only be used in Months 1, 2 or 3, and must be thrown away at the end of Month 3; in other words, oil bought in Month 1 cannot be used in Month 4, 5, ... An analogous statement holds for oil bought in other months.** 

Formulate a linear optimization problem to decide how much canola oil to buy for each of the six months in order to minimize the total purchase cost over these six months. 

**Decision Variables:** 





**Objective:**






**Constraints:**







In [1]:
import pandas as pd
from gurobi import Model, GRB

In [7]:
price = [150,160,180,170,180,160]
demand = [2000,2500,3000,3000,2500,2000]

In [22]:
mod = Model()
x = mod.addVars(range(1,7),name = 'x')
y = mod.addVars(range(1,7),name = 'y')
mod.setObjective(sum(price[i-1]*x[i] for i in range(1,7)),sense = GRB.MINIMIZE)
mod.addConstrs((y[i] == x[i] - demand[i-1] if i == 1 else y[i] == x[i] + y[i-1] - demand[i-1] 
                if i == 2 else y[i] == x[i] + x[i-1] + x[i-2] - demand[i-1] for i in range(1,7)), name = 'balance')
mod.addConstrs((x[i] >= demand[i-1] if i == 1 else x[i] + y[i-1] >= demand[i-1] 
                for i in range(1,7)), name = 'demand')
mod.addConstrs((y[i] <= 8000 
                for i in range(1,7)), name = 'storage')

# mod.update()
# mod.write('ProblemB.lp')
# %cat 'ProblemB.lp'

mod.setParam('OutputFlag',False)
mod.optimize()

print(mod.ObjVal)
for i in range(1,7):
    print(f'x[{i}]: {x[i].x}')
    print(f'y[{i}]: {y[i].x}')

1460000.0
x[1]: 2000.0
y[1]: 0.0
x[2]: 2500.0
y[2]: 2000.0
x[3]: 1000.0
y[3]: 2500.0
x[4]: 2000.0
y[4]: 2500.0
x[5]: 0.0
y[5]: 500.0
x[6]: 1500.0
y[6]: 1500.0


In [25]:
mod = Model()
x = mod.addVars(range(1,7),name = 'x')
y = mod.addVars(range(1,7),name = 'y')
z = mod.addVars(range(1,7),name = 'z')
mod.setObjective(sum(price[i-1]*x[i] for i in range(1,7)),sense = GRB.MINIMIZE)
mod.addConstrs((x[i]-y[i]-z[i] == demand[i-1] if i == 1 else x[i]-y[i]-z[i]+y[i-1] == demand[i-1] 
                if i == 2 else x[i]-y[i]-z[i]+y[i-1]+z[i-2] == demand[i-1] for i in range(1,7)), name = 'balance')
# mod.addConstrs((x[i] >= demand[i-1] if i == 1 else x[i] + y[i-1] >= demand[i-1] 
#                 for i in range(1,7)), name = 'demand')
mod.addConstrs((y[i] + z[i] <= 8000 if i == 1 else y[i] + z[i] + z[i-1] <= 8000
                for i in range(1,7)), name = 'storage')

# mod.update()
# mod.write('ProblemB.lp')
# %cat 'ProblemB.lp'

mod.setParam('OutputFlag',False)
mod.optimize()

print(mod.ObjVal)
for i in range(1,7):
    print(f'x[{i}]: {x[i].x}')
    print(f'y[{i}]: {y[i].x}')

2305000.0
x[1]: 10000.0
y[1]: 8000.0
x[2]: 2500.0
y[2]: 8000.0
x[3]: 0.0
y[3]: 5000.0
x[4]: 500.0
y[4]: 2500.0
x[5]: 0.0
y[5]: 0.0
x[6]: 2000.0
y[6]: 0.0
